In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [11]:
def cosine_similarity(A, B):

    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)
    return cos

In [12]:
# UNQ_C8 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def nearest_neighbor(v, candidates, k=1, cosine_similarity=cosine_similarity):
    similarity_l = []
    for row in candidates:
        cos_similarity = cosine_similarity(v,row)
        similarity_l.append(cos_similarity)

    sorted_ids = np.argsort(similarity_l)  
    sorted_ids = sorted_ids[::-1]
    k_idx = sorted_ids[:k]

    return k_idx

In [13]:
def HashVal(v,planes):
    signs = np.sign(np.dot(planes.T,v))
    bool_val = [1 if i>0 else 0 for i in signs]
    hash_val = 0
    for i in range(len(bool_val)):
        hash_val+=2**i*hash_val[i]
    hash_val = int(hash_val)
    return hash_val

In [14]:
# planes are randomly generated plane vectors that must separate space
# planes divide space into several parts that are called Bucket
def HashTable(vecs, planes):
    nplanes = planes.shape[1]
    nbuckets = 2**nplanes
    HashTable = {i:[] for i in range(nbuckets)}
    IdTable = {i:[] for i in range(nbuckets)}
    
    for i,v in enumerate(vecs):
        H = HashVal(v,planes)
        HashTable[H].append(v)
        IdTable[H].append(i)
        
    return HashTable,IdTable

In [15]:
def approximate_knn(doc_id, v, planes_l, hash_tables, id_tables, k=1, num_plane_multiverse=25, HashVal=HashVal):
    
    vecs = list()
    ids = list()
    ids_set = set()

    for universe_id in range(num_plane_multiverse):
        # plane Multiverse; ამ შემთხვევაში რენდომად ხდება 25 ჯერ ვექტორული სივრცის დაყოფა planes_l[universe_id].shape[1] სიბრტყით დაყოფა
        planes = planes_l[universe_id]
        #ითვლება ჩვენი გადაცემული ვექტორისათვის hash
        hash_value = hash_value_of_vector(v, planes)
        #ვიძახებთ ამ კონკრეტულ universe-ში არსებულ hash_table-ს
        hash_table = hash_tables[universe_id]
        # და ამ universe-ში ვიღებთ ამ კონკრეტულ hash-ში არსებულ სიტყვების hash-მნიშვნელობებს
        document_vectors_l = hash_table[hash_value]
        # ასევე გვჭირდება hash_table-ის ID
        id_table = id_tables[universe_id]
        # ვიღებთ იმ ID ებს რომლებიც ამ Universe-ში უნდა იყოს ჩვენი გადაცემული ვექტორის მეზობელი:
        new_ids_to_consider = id_table[hash_value]

        if doc_id in new_ids_to_consider:
            new_ids_to_consider.remove(doc_id)
        for i, new_id in enumerate(new_ids_to_consider):
            if new_id not in ids_set:
                document_vector_at_i = document_vectors_l[i]

                vecs_to_consider_l.append(document_vector_at_i)
                ids_to_consider_l.append(new_id)
                ids_to_consider_set.add(new_id)

    print("Fast considering %d vecs" % len(vecs_to_consider_l))

    vecs_to_consider_arr = np.array(vecs_to_consider_l)

    nearest_neighbor_idx_l = nearest_neighbor(v, vecs_to_consider_arr, k=k)
    nearest_neighbor_ids = [ids_to_consider_l[idx]
                            for idx in nearest_neighbor_idx_l]

    return nearest_neighbor_ids
    